importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
import tensorflow_datasets as tfds

In [7]:
from tensorflow.keras import Model

Configurations

In [2]:
epochs= 4
batch_size = 100
learning_rate=0.0001
momentum=0.5
log_interval=10
random_seed=1

In [3]:
tf.random.set_seed(random_seed)

loading MNIST dataset

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

11490434/11490434 [==============================] - 1s 0us/step


slice and load the dataset

In [5]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Creating the model and forward-forward algorithm

In [8]:
class FF(Model):
  def __init__(self):
    super(FF, self).__init__()
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation="softmax")
    self.params = tf.Variable(tf.zeros((1,), dtype=tf.bool), trainable=False)

  def call(self, x):
    x = self.flatten(x)
    x = self.d1(x)
    x= self.d2(x)
    self.params.assign(tf.logical_not(self.params))
    print(self.params)
    print(x.shape)
    return x

# Create an instance of the model
model = FF()

One forward pass

In [13]:
example = iter(train_ds)
example_data, example_targets = next(example)
model(example_data)

<tf.Variable 'Variable:0' shape=(1,) dtype=bool, numpy=array([ True])>
(32, 10)


<tf.Tensor: shape=(32, 10), dtype=float32, numpy=
array([[0.03973895, 0.13786587, 0.03795836, 0.0958503 , 0.14267398,
        0.0718351 , 0.04977502, 0.12293228, 0.21315588, 0.08821434],
       [0.07342072, 0.08494646, 0.06460295, 0.10433173, 0.1321928 ,
        0.08682901, 0.08657654, 0.15618183, 0.12479551, 0.08612233],
       [0.095995  , 0.12179304, 0.07758217, 0.11635479, 0.1582381 ,
        0.05258871, 0.08364556, 0.1494798 , 0.09762781, 0.04669501],
       [0.17337435, 0.07931091, 0.07195922, 0.06100505, 0.16529971,
        0.05999302, 0.12344603, 0.10021453, 0.11974315, 0.04565393],
       [0.0806116 , 0.11982178, 0.09960887, 0.09154337, 0.12207858,
        0.07507015, 0.05141668, 0.08825354, 0.17583914, 0.09575628],
       [0.15078259, 0.11166342, 0.06791545, 0.09723182, 0.09309863,
        0.11769054, 0.05890414, 0.13641271, 0.1080145 , 0.05828623],
       [0.0680603 , 0.15980725, 0.04850465, 0.0837505 , 0.18483807,
        0.05786558, 0.09053875, 0.12459241, 0.12998839, 0.05

printing the model

In [16]:
print(model)

Define loss and optimizer func

In [14]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)

defining an accuracy metric

In [15]:
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()